In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from catboost import CatBoostClassifier
import janestreet

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

##Pandas powered by Cuda
import cudf

df_train_cuda = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
df_train = df_train_cuda.to_pandas()
del df_train_cuda

features_meta_cudf = cudf.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
features_meta = features_meta_cudf.to_pandas()
del features_meta_cudf

print(f'train shape: {format(df_train.shape)}')
print(f'features_meta shape: {format(features_meta.shape)}')

features = [col for col in list(df_train.columns) if 'feature' in col]

df_train = df_train[df_train['weight'] != 0]

df_train['action'] = (df_train['resp'].values > 0).astype(int)

NAN_VALUE = -999

f_mean = df_train.mean()
df_train.fillna(f_mean)

X_train = df_train.loc[:, features]
y_train = df_train.loc[:, 'action']

del df_train
gc.collect()

print('Creating classifier...', end='')
clf = CatBoostClassifier(loss_function = 'Logloss',
                         task_type="GPU",
                         learning_rate = 0.1)


clf.fit(X_train, y_train)

print('Finished.')

del X_train, y_train
gc.collect()

env = janestreet.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:    
    test_weight = test_df.iloc[0].weight
    if test_weight > 0:
        sample_prediction_df.action = clf.predict(test_df.loc[:, features].fillna(NAN_VALUE))[0]
    else:
        sample_prediction_df.action = 0
    env.predict(sample_prediction_df)

In [ ]:
#df_train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
#features_meta = pd.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')

#print(f'train shape: {format(df_train.shape)}')
#print(f'features_meta shape: {format(features_meta.shape)}')

In [ ]:
#df_train = df_train.sample(5000)